Goal:
===

I'm going to derive the equations for computing probabilities on hidden states given observed data for [Hidden Markov Models](https://en.wikipedia.org/wiki/Hidden_Markov_model)



# Preliminaries:

I'm going to define my variables here:

* $(Y_t)_{t=0}^{T}$ is the observed sequence
* $(X_t)_{t=0}^T$ is the hidden sequence.
* $A_{i\rightarrow j} := Pr(X_{t+1} = j| X_t = i)$ is the transition matrix from one hidden state to another.
* $B_i(j) := Pr(Y = j | X = i)$ is the probability distribution of the observed value as a function of the hidden state.
* $\pi(i) := Pr(X_0 = i)$ is the initial probability distribution on the hidden state.


Here are the assumptions based on the Markovity of the hidden sequence:

$$Pr(X_{t+1} = j| Y_0, \ldots, Y_t, X_0=s_0,X_1=s_1, \ldots,X_t = s_t) = Pr(X_{t+1} = j | X_t = s_t)$$

and

$$Pr((Y_s)_{s=t+1}^T | (Y_s)_{s=0}^t,X_0 = s_0,\ldots,X_t=s_t) = Pr((Y_s)_{s=t}^T | X_t=s_t)$$

# The Goals:
To be able to compute:
    
* $Pr(X_t = i | (Y_s)_{s=0}^T)$
* $Pr(X_{t} =i, X_{t+1} = j | (Y_s)_{s=0}^T)$
* Find a better model $\pi, A, B$ that explains the observed data.
* For a given set of parameters $\pi, A, B$ find the most probable sequence of hidden states $(X_t)$ given
the observed sequence $(Y_t)$.

To do so I'll need to do some recursive computations. We begin:

## Define: 

* $\gamma_{t}(i) := Pr((Y_s)_{s=0}^T, X_t = i)$
* $\alpha_t(i) := Pr((Y_s)_{s=0}^t, X_t = i)$
* $\beta_{t}(i) := Pr((Y_s)_{s=t+1}^T| X_t=i)$

## Relationship between $\alpha, \beta, \gamma$:
Here's the relationship between $\alpha_t, \beta_t, \gamma_t$ :

\begin{align}
\gamma_{t}(i) &:= Pr((Y_s)_{s=0}^T, X_t = i) \\
 &= Pr((Y_s)_{t+1}^T | X_t = i, (Y_s)_{s=0}^t ) Pr(X_t = i, (Y_s)_{s=0}^t )\\
 &= Pr((Y_s)_{t+1}^T | X_t = i) Pr(X_t = i, (Y_s)_{s=0}^t )\\
 &= \beta_t(i)\alpha_t(i) \\
 \gamma_{t}(i) &= \beta_t(i)\alpha_t(i) \\
\end{align}  

Since $\gamma_T(i) \equiv \alpha_T(i)$ then $\beta_T(i) = 1$


Here's the relationship between $\alpha_t, \alpha_{t+1}$: 

\begin{align}
\alpha_{t+1}(j) &= Pr((Y_s)_{s=0}^{t+1}, X_{t+1} = j) \\
&= \sum_i Pr( (Y_s)_{s=0}^{t+1}, X_{t+1} = j, X_t = i) \\
&= \sum_i Pr(Y_{t+1}, X_{t+1} = j | (Y_s)_{s=0}^{t}, X_t = i)Pr( (Y_s)_{s=0}^{t}, X_t = i) \\
&= \sum_i Pr(Y_{t+1}, X_{t+1} = j | X_t = i)\alpha_t(i) \\
&= \sum_i Pr(Y_{t+1}|X_{t+1} = j, X_t = i)Pr(X_{t+1} = j| X_t = i)\alpha_t(i) \\
&= \sum_i Pr(Y_{t+1}|X_{t+1} = j)A_{i\rightarrow j}\alpha_t(i) \\
&= \sum_i B_j(Y_{t+1})A_{i\rightarrow j}\alpha_t(i) \\
\alpha_{t+1}(j) &= \sum_i B_j(Y_{t+1})A_{i\rightarrow j}\alpha_t(i) \\
\end{align}


Here's the relationship between $\beta_t$ and $\beta_{t+1}$:

\begin{align}
\beta_t(j) & = Pr((Y_s)_{s=t+1}^T| X_t=j) \\
&= \sum_i Pr((Y_s)_{s=t+1}^T,X_{t+1} = i| X_t=j) \\
&= \sum_i Pr((Y_s)_{s=t+2}^T|Y_{t+1},X_{t+1} = i, X_t=j) Pr(Y_{t+1},X_{t+1} = i| X_t = j)\\
&= \sum_i Pr((Y_s)_{s=t+2}^T|X_{t+1} = i) Pr(Y_{t+1},X_{t+1} = i| X_t = j)\\
&= \sum_i \beta_{t+1}(i) Pr(Y_{t+1},X_{t+1} = i| X_t = j)\\
&= \sum_i  \beta_{t+1}(i)Pr(Y_{t+1}|X_{t+1} = i, X_t = j)Pr(X_{t+1} = i | X_t = j)\\
&= \sum_i  \beta_{t+1}(i)Pr(Y_{t+1}|X_{t+1} = i)A_{j\rightarrow i}\\
&= \sum_i  \beta_{t+1}(i)B_i(Y_{t+1})A_{j\rightarrow i}\\
\beta_t(j) &= \sum_i  \beta_{t+1}(i)B_i(Y_{t+1})A_{j\rightarrow i}\\
\end{align}

# Scaling Factors
For any decent length observed sequence, the $\alpha, \beta, \gamma$ sequences will quickly converge to 0 due to underflow issues in the finite precision arithmetic we have for computations.

To avoid this underflow we instead compute conditional probabilities recursively which will not underflow as follows:

## Conditional $\alpha$
\begin{align}
\hat{\alpha}_t(i) &:= Pr(X_t = i | Y_0,\ldots, Y_t) \\
&= \frac{Pr(X_t = i, Y_0,\ldots, Y_t)}{Pr(Y_0,\ldots, Y_t)} \\
&= \frac{\alpha_t(i)}{\Lambda_t}, \quad \Lambda_t := Pr(Y_0,\ldots, Y_t)
\end{align}
$\hat{\alpha}_t$ satisfies the equation:

$$\sum_i \hat{\alpha}_t(i) = \sum_i Pr(X_t = i | Y_0,\ldots, Y_t) = 1$$
In terms of the recursive computation of $\alpha_{t+1}$  as a function of $\alpha_t$ we have the following:

\begin{align}
\alpha_{t+1}(j) &= \sum_i B_j(Y_{t+1})A_{i\rightarrow j}\alpha_t(i) \\
\Lambda_{t+1}\hat{\alpha}_{t+1}(j) &= \sum_i B_j(Y_{t+1})A_{i\rightarrow j}\Lambda_t\hat{\alpha}_t(i) \\
\hat{\alpha}_{t+1}(j) &= \frac{\Lambda_{t}}{\Lambda_{t+1}}\sum_i B_j(Y_{t+1})A_{i\rightarrow j}\hat{\alpha}_t(i) \\
&= \frac1{\sigma_{t+1}} \sum_i B_j(Y_{t+1})A_{i\rightarrow j}\hat{\alpha}_t(i), \quad \sigma_{t+1} := \frac{\Lambda_{t+1}}{\Lambda_{t}}
\end{align}
We define $\sigma_0 := Pr(Y_0)$, and we can recover $\sigma_{t+1}$ recursively from the condition that $\sum_j \hat{\alpha}_{t+1}(j) = 1$ which implies:

$$\sigma_{t+1} = \sum_j\sum_i B_j(Y_{t+1})A_{i\rightarrow j}\hat{\alpha}_t(i)$$ 

and also:

$$\Lambda_t = \prod_{s=0}^t \sigma_t$$

# Conditional $\beta$

We want to take advantage of how we defined $\hat{\alpha}$ so we can compute a $\beta$ that does not underflow. Let's define $\hat{\gamma}_t(i)$ first:

\begin{align}
\hat{\gamma}_t(i) &:= Pr(X_t = i | Y_0,\ldots, Y_T) \\
&= \frac{Pr(X_t = i , Y_0,\ldots, Y_T)}{Pr(Y_0,\ldots, Y_T)} \\
&= \frac{\alpha_t(i)\beta_t(i)}{Pr(Y_0,\ldots, Y_T)} \\
&= \frac{\Lambda_t\hat{\alpha}_t(i)\beta_t(i)}{Pr(Y_0,\ldots, Y_T)} \\
&= \hat{\alpha}_t(i)\beta_t(i)\frac{\Lambda_t}{Pr(Y_0,\ldots, Y_T)} \\
&= \hat{\alpha}_t(i)\hat{\beta}_t(i) \\
\end{align}
Now we can define $\hat{\beta}_t(i)$:
\begin{align}
\hat{\beta}_t(i) &:= \frac{\Lambda_t}{Pr(Y_0,\ldots, Y_T)}\beta_t(i) \\
&= \frac{\prod_{s=0}^t \sigma_s}{\prod_{s=0}^T \sigma_s}\beta_t(i)\\
&= \frac1{\prod_{s=t+1}^T \sigma_s}\beta_t(i)
\end{align}

Since $\Lambda_{T} = Pr(Y_0,\ldots,Y_T)$ we can deduce that $\hat{\beta}_T(i) = \beta_T(i) = 1$ 

Now we can take advantage of the recursion on $\beta_t$ to compute $\hat{\beta}_t$ as a function of $\hat{\beta}_{t+1}$:

\begin{align}
\beta_t(j) &= \sum_i  \beta_{t+1}(i)B_i(Y_{t+1})A_{j\rightarrow i}\\
\left(\prod_{s=t+1}^T \sigma_s\right) \hat{\beta}_t(j) &= \left(\prod_{s=t+2}^T\sigma_s\right)\sum_i  \hat{\beta}_{t+1}(i)B_i(Y_{t+1})A_{j\rightarrow i}\\
\hat{\beta}_t(j) &=\frac{1}{\sigma_{t+1}}\sum_i  \hat{\beta}_{t+1}(i)B_i(Y_{t+1})A_{j\rightarrow i}\\
\end{align}

# Updating the estimates of $A,B,\pi$ based on the observations
I want to apply [Baum-Welch algorithm](https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm) to improve the model for the observed data by updating the parameters $A,B,\pi$. Some of the parameters are easy to re-estimate:

$\pi(i) := Pr(X_0 = i)$ gets updated to $\pi'(i) = Pr(X_0=i | Y_0,\ldots, Y_T) = \hat{\gamma}_0(i)$

$B_i(y) := Pr(Y=y|X = i)$ gets updated to 

\begin{align}
B'_i(y) &:= Pr(Y=y | X=i, Y_0,\ldots, Y_T) \\
&= \frac{Pr(Y=y, X=i, Y_0,\ldots, Y_T)}{Pr(X=i, Y_0,\ldots, Y_T)} \\
&= \frac{Pr(Y=y, X=i| Y_0,\ldots, Y_T)Pr( Y_0,\ldots, Y_T)}{Pr(X=i| Y_0,\ldots, Y_T)Pr( Y_0,\ldots, Y_T)} \\
&= \frac{Pr(Y=y, X=i| Y_0,\ldots, Y_T)}{Pr(X=i| Y_0,\ldots, Y_T)} \\
&= \frac{Pr(X=i| Y=y, Y_0,\ldots, Y_T)Pr(Y=y|Y_0,\ldots, Y_T)}{Pr(X=i| Y_0,\ldots, Y_T)} \\
&= \frac{Pr(X=i| Y=y, Y_0,\ldots, Y_T)Pr(Y=y|Y_0,\ldots, Y_T)}{\frac1T\sum_{t=0}^T Pr(X_t=i| Y_0,\ldots, Y_T)} \\
&= \frac{Pr(X=i| Y=y, Y_0,\ldots, Y_T)Pr(Y=y|Y_0,\ldots, Y_T)}{\frac1T\sum_{t=0}^T\hat{\gamma}_t(i)} \\
&= \frac{\frac1T\sum_{t=0,Y_t=y}^T Pr(X=i| Y_0,\ldots, Y_T)\times 1}{\frac1T\sum_{t=0}^T\hat{\gamma}_t(i)} \\
&= \frac{\sum_{t=0,Y_t=y}^T \hat{\gamma}_t(i)}{\sum_{t=0}^T\hat{\gamma}_t(i)} \\ 
\end{align}

Now I want to determine how to update $A$:

$A_{i\rightarrow j} = Pr(X_{new} = j | X_{old} = i)$ updates to 

\begin{align}
A'_{i \rightarrow j} &= Pr(X_{new} = j | X_{old} = i,Y_0,\ldots, Y_T) \\
&= \frac{Pr(X_{new} = j, X_{old} = i,Y_0,\ldots, Y_T)}{Pr(X_{old} = i,Y_0,\ldots, Y_T)} \\
&= \frac{\frac1{T-1}\sum_{t=0}^{T-1} Pr(X_{t+1} = j, X_{t} = i,Y_0,\ldots, Y_T)}{Pr(X_{old} = i,Y_0,\ldots, Y_T)} \\
&= \frac{\frac1{T-1}\sum_{t=0}^{T-1} Pr(X_{t+1} = j, Y_{t+1},\ldots Y_T| X_{t} = i,Y_0,\ldots, Y_t)Pr(X_{t} = i,Y_0,\ldots, Y_t)}{Pr(X_{old} = i,Y_0,\ldots, Y_T)} \\
&= \frac{\frac1{T-1}\sum_{t=0}^{T-1} Pr(Y_{t+1},\ldots Y_T| X_{t+1} = j, X_{t} = i,Y_0,\ldots, Y_t)Pr(X_{t+1} = j| X_{t} = i,Y_0,\ldots, Y_t) Pr(X_{t} = i,Y_0,\ldots, Y_t)}{Pr(X_{old} = i,Y_0,\ldots, Y_T)} \\
&= \frac{\frac1{T-1}\sum_{t=0}^{T-1} Pr(Y_{t+1},\ldots Y_T| X_{t+1} = j)Pr(X_{t+1} = j| X_{t} = i) Pr(X_{t} = i,Y_0,\ldots, Y_t)}{Pr(X_{old} = i,Y_0,\ldots, Y_T)} \\
&= \frac{\frac1{T-1}\sum_{t=0}^{T-1} Pr(Y_{t+2},\ldots Y_T| X_{t+1} = j,Y_{t+1})Pr(Y_{t+1}|X_{t+1}=j)A_{i\rightarrow j}\alpha_t(i)}{Pr(X_{old} = i,Y_0,\ldots, Y_T)} \\
&= \frac{\frac1{T-1}\sum_{t=0}^{T-1} \beta_{t+1}(j)B_j(Y_{t+1})A_{i\rightarrow j}\alpha_t(i)}{Pr(X_{old} = i,Y_0,\ldots, Y_T)} \\
&= \frac{\frac1{T-1}\sum_{t=0}^{T-1} \left(\prod_{s=t+2}^T \sigma_s\right) \hat{\beta}_{t+1}(j)B_j(Y_{t+1})A_{i\rightarrow j}\left(\prod_{s=0}^t \sigma_s\right)\hat{\alpha}_t(i)}{Pr(X_{old} = i,Y_0,\ldots, Y_T)} \\
&= \frac{\frac1{T-1}\sum_{t=0}^{T-1} \left(\prod_{s\ne t+1}^T \sigma_s\right) \hat{\beta}_{t+1}(j)B_j(Y_{t+1})A_{i\rightarrow j}\hat{\alpha}_t(i)}{\frac1{T-1}\sum_{t=0}^{T-1}Pr(X_t = i,Y_0,\ldots, Y_T)} \\
&= \frac{\frac1{T-1}\sum_{t=0}^{T-1} \left(\prod_{s\ne t+1}^T \sigma_s\right) \hat{\beta}_{t+1}(j)B_j(Y_{t+1})A_{i\rightarrow j}\hat{\alpha}_t(i)}{\frac1{T-1}\sum_{t=0}^{T-1}Pr(X_t = i|Y_0,\ldots, Y_T)Pr(Y_0,\ldots, Y_T)} \\
&= \frac{\sum_{t=0}^{T-1} \left(\prod_{s\ne t+1}^T \sigma_s\right) \hat{\beta}_{t+1}(j)B_j(Y_{t+1})A_{i\rightarrow j}\hat{\alpha}_t(i)}{\sum_{t=0}^{T-1}\hat{\gamma}_t(i)Pr(Y_0,\ldots, Y_T)} \\
&= \frac{\sum_{t=0}^{T-1} \left(\prod_{s\ne t+1}^T \sigma_s\right) \hat{\beta}_{t+1}(j)B_j(Y_{t+1})A_{i\rightarrow j}\hat{\alpha}_t(i)}{\sum_{t=0}^{T-1}\hat{\gamma}_t(i)\prod_{s=0}^T\sigma_s} \\
&= \frac{\sum_{t=0}^{T-1}  1/\sigma_{t+1} \hat{\beta}_{t+1}(j)B_j(Y_{t+1})A_{i\rightarrow j}\hat{\alpha}_t(i)}{\sum_{t=0}^{T-1}\hat{\gamma}_t(i)} \\
\end{align}



# Computing $\log\left(Pr(Y_0,\ldots, Y_T| \pi,A,B)\right)$
Here's how we compute the log of the probability $Pr(Y_0,\ldots, Y_T| \pi,A,B)$:

\begin{align}
Pr(Y_0,\ldots, Y_T| \pi,A,B) &= \prod_{t=0}^T \sigma_t \\
\log\left(Pr(Y_0,\ldots, Y_T| \pi,A,B)\right) &= \sum_{t=0}^T \log\left(\sigma_t\right) \\
\end{align}

## Determining the Most Probable Path
Once the parameters $\pi, A, B$ have been set and given the observed sequence $Y_0,\ldots,Y_t$ how does one compute the most probable sequence of hidden states $X_0,\ldots, X_T$? The technique we use is the [Viterbi algorithm](https://en.wikipedia.org/wiki/Viterbi_algorithm) or dynamic programming.


Let $(n_t)_{t=1}^T$ denote a possible sequence of hidden states, $X_t = n_t$. How do we compute the probability of this occurrence?

\begin{align}
Pr((n_t)|\pi,A,B,(Y_t)) &= \frac{Pr((n_t),\pi,A,B,(Y_t))}{Pr(\pi,A,B,(Y_t))}\\
&= \frac{Pr((n_t)_{t=2}^T,\pi,A,B,(Y_t)_{t=2}^T)|Y_1,n_1)Pr(Y_1,n_1)}{Pr(\pi,A,B,(Y_t))}\\
&= \frac{Pr((n_t)_{t=2}^T,\pi,A,B,(Y_t)_{t=2}^T)|Y_1,n_1)Pr(Y_1|n_1)Pr(n_1)}{Pr(\pi,A,B,(Y_t))}\\
&= \frac{B(n_1,Y_1)\pi(n_1)\prod_{t=2}^T A_{n_{t-1} \rightarrow n_t}B(n_t,Y_t)}{Pr(\pi,A,B,(Y_t))}\\
\end{align}

**Algorithm**
To compute the most probable path we recursively compute the most probable path to time $t$.

1. We have $N$ hidden states.
2. For a time $t$ if we can find the $N$ most probable paths $p_{t,n}$ to the hidden state $X_t = 1, \ldots, N$ we can extend them to the $N$ most probable paths $p_{t+1,n}$.



$$p_{t+1,n} = \max\arg_{i}\{p_{t,i}A_{i\rightarrow n}B(n,Y_{t+1}) \}$$
3. Begin with $t=0$, $(p_{0,n} := \pi(n)B(n,Y_0))_{n=1}^N$ and recurse through $t$ until we get to $t=T$ then the
most probable path is given by $\max arg_{i} \{ p_{T,i} \}$

# Determining the number of hidden states
I can use [Wilks' theorem](https://en.wikipedia.org/wiki/Wilks%27_theorem) to determine the number of hidden states as follows:

1. Assume  $n$ hidden states. A model with $n$ hidden states and $S$
 observable states requires $n-1 + n(S-1) + n(n-1) = n(1 + S-1 + n -1) -1 = n(n + S -1) - 1 $ parameters.
2. Use Baum-Welch's algorithm to hill climb to the best model it can find. 
3. Compute the log likelihood of the observed data , $S_{null} :=\log Pr(Y_0,\ldots, Y_T| \pi, A,B)$.
4. Assume now $n+1$ hidden states and as above recover the MLE parameters and  log likelihood, $S_{alternative}$.
5. Compute the statistic $\Lambda := 2\times (S_{alternative} - S_{null})$. This statistic will be a sample from a Chi-squared distribution with $df = (n+1)(n+1 +S -1) - n(n+S-1) = n^2 + n + nS + S - n^2 - nS + n = 2n +S$ degrees of freedom. 
6. Compute the $p$ value $p:=Pr(\xi \ge \Lambda)$. where $\xi$ is a random variable of type Chi-Square distribution with $df = 2n +S$ degrees of freedom. If $p$ is below my threshold stick with alternative hypothesis (1 more hidden state) otherwise stop at $n$ hidden states.


# Demonstration
I want to demonstrate that this derivation is correct.

In [ ]:
%pylab inline

In [ ]:
import numba
import scipy.stats as st
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def make_random(N,S):
    """generates a random pi,A,B triplet for me."""

    pi = np.random.dirichlet(np.ones(N))
    A  = np.random.dirichlet(np.ones(N),N).transpose()
    B  = np.random.dirichlet(np.ones(S),N)

    return pi,A,B

In [ ]:
def generate_data(pi,A,B,T = 100):
    """Generates an observed sequence of length T"""
    N, S = B.shape
    
    # Generate hidden path
    X_t = np.zeros(T,dtype=np.int32)
    X_t[0] = np.argwhere(np.random.multinomial(1,pi,size=1).flatten()==1).flatten()[0]
    for t in range(1,T):
        X_t[t] = np.argwhere(np.random.multinomial(1,A[:,X_t[t-1]],size=1).flatten()==1).flatten()[0]

    # Generate observed sequence
    Y_t = np.zeros(T,dtype=np.int32)
    for t in range(T):
        Y_t[t] = np.argwhere(np.random.multinomial(1,B[X_t[t]],size=1).flatten()==1).flatten()[0]

    return X_t,Y_t

In [ ]:
@numba.njit()
def alphapass(alpha,sigma,A,B,pi,observed):
    """
    alpha(t,i) = Pr(hidden_t = i | obs_1,...,obs_t)
    beta(t,i)  = Pr(obs_t+1,...,obs_T| hidden_t = i)*Pr(o_1,...,o_t)/Pr(o_1,...,o_T)
    gamma(t,i)  = Pr(hidden_t = i | obs_1,...,obs_T)
    A(j,i) == Pr( hidden i -> hidden j)
    B(i,j) == Pr( observed j | hidden i)
    """

    N,S = B.shape
    T = observed.shape[0]

    # initial iteration
    alpha[0] = pi
    alpha[0] = alpha[0]*B[:,observed[0]]
    sigma[0] = alpha[0].sum()
    alpha[0] /= sigma[0]

    # now do the remainder
    for t in range(1,T):
        alpha[t] = np.dot(A,alpha[t-1])
        alpha[t] *= B[:,observed[t]]
        sigma[t] = alpha[t].sum()
        alpha[t] /= sigma[t]

In [ ]:
@numba.njit()
def betapass(beta,sigma,A,B,observed):
    """
    A(j,i) == Pr( hidden i -> hidden j)
    B(i,j) == Pr( observed j | hidden i)
    """

    N,S = B.shape
    T = observed.shape[0]

    At = A.transpose()

    # initial iteration
    beta[-1] = 1.

    # now do the remainder
    for t in range(T-1,0,-1):
        beta[t-1] = np.dot(At,beta[t]*B[:,observed[t]])/sigma[t]

In [ ]:
@numba.njit()
def gammapass(alpha,beta,gamma):
    gamma[:,:] = alpha*beta

In [ ]:
@numba.njit()
def update_B(B,gamma, observed):
    """ Updates belief on B """
    N, S = B.shape
    T = observed.shape[0]

    B_new = np.zeros((N,S))

    for t in range(T):
        B_new[:,observed[t]] += gamma[t]

    B_new = B_new/B_new.sum(axis=1).reshape(-1,1)
    B[:] = B_new

In [ ]:
def update_pi(pi,gamma):
    """ Updates belief on pi """
    pi[:] = gamma[0]

In [ ]:
@numba.njit()
def digammapass(alpha,beta,gamma,observed,sigma,digamma,A,B):
    """
    Computes an updated version of A based on the observed sequence of data.
    
    alpha(t,i) = Pr(hidden_t = i | obs_1,...,obs_t)
    beta(t,i)  = Pr(obs_t+1,...,obs_T| hidden_t = i)*Pr(o_1,...,o_t)/Pr(o_1,...,o_T)
    gamma(t,i)  = Pr(hidden_t = i | obs_1,...,obs_T)
    A(j,i) = Pr( hidden i -> hidden j)
    B(i,j) = Pr( observed j | hidden i)

    Reestimates Pr(X=i -> X=j) via averaging
    Pr(x_t=i,x_t+1=j|o_1,...,o_T) = alpha_t(i)A_jibeta_{t+1}(j)B(j,o_{t+1})
    """

    T = alpha.shape[0]
    N,S = B.shape
    
    digamma[:] = 0
    digammatemp = np.zeros((N,N))
    for t in range(0,T-1):
        temp_i = (alpha[t]/sigma[t+1]).reshape(1,-1)*A
        temp_j = (B[:,observed[t+1]]*beta[t+1]).reshape(-1,1)
        digammatemp = temp_i*temp_j
        digamma[:] += digammatemp

    # rescale columns as \sum_j A_{j,i} == 1
    digamma[:] = digamma/digamma.sum(axis=0).reshape(1,-1)

In [ ]:
def update_parameters(alpha,beta,gamma, pi, A,B,observed,sigma):
    """ Updates pi, A,B from observed data. """

    N,S = B.shape
    digamma = np.zeros((N,N))
    digammapass(alpha, beta, gamma, observed,sigma,digamma, A,B)

    update_B(B,gamma,observed)
    update_pi(pi,gamma)

    A[:] = digamma[:]

In [ ]:
@numba.njit()
def gammaoneshot(alpha,beta,gamma,pi,A,B,observed,sigma):
    """ computes alpha, beta, and gamma directly for me. """

    alphapass(alpha,sigma,A,B,pi,observed)
    betapass(beta,sigma,A,B,observed)
    gammapass(alpha,beta,gamma)

In [ ]:
def compute_loglikelihood(sigma):
    """Computes Pr(Y_0,...,Y_T| pi, A, B) from (sigma_t)"""
    return log(sigma).sum()

def compute_loglikelihood_parameters(pi,A,B,observed):
    """Computes Pr(Y_0,...,Y_T| pi, A, B) from (pi, A, B)"""
    N, S = B.shape
    T = observed.shape[0]
    alpha = zeros((T,N))
    beta  = zeros((T,N))
    gamma = zeros((T,N))
    sigma = zeros(T)
    
    gammaoneshot(alpha,beta,gamma,pi,A,B,observed,sigma)
    score = compute_loglikelihood(sigma)
    return score

In [ ]:
def reestimate_parameters(A,B,pi,observed,
                          halting_criteria=1e-6,debug=False):
    """re-estimates pi, A, and B from the observed sequence."""
    N, S = B.shape
    T = observed.shape[0]
    alpha = zeros((T,N))
    beta  = zeros((T,N))
    gamma = zeros((T,N))
    sigma = zeros(T)
    
    current_score = -np.inf
    gammaoneshot(alpha,beta,gamma,pi,A,B,observed,sigma)
    new_score = compute_loglikelihood(sigma)
    diff = new_score - current_score
    while diff > halting_criteria:
        current_score = new_score
        update_parameters(alpha,beta,gamma, pi, A,B,observed,sigma)
        gammaoneshot(alpha,beta,gamma,pi,A,B,observed,sigma)
        new_score = compute_loglikelihood(sigma)
        diff = new_score - current_score
        if debug:
            print("diff = ", diff)

In [ ]:
def determine_number_of_hidden_states(Y_t,halting_criteria=1e-6,verbose=False):
    """Uses Wilks' theorem to determine the number of hidden states based on
    the observed data (Y_t). It returns the number of hidden states"""
    S = Y_t.max() + 1
    N = 1
    my_pi, my_A, my_B = make_random(N,S)

    reestimate_parameters(my_A,my_B, my_pi,Y_t,halting_criteria=halting_criteria)
    S_alternative =compute_loglikelihood_parameters(my_pi, my_A, my_B, Y_t)

    p = 0.0
    while p < 0.01:
        df = 2*N + S
        N += 1
        my_pi, my_A, my_B = make_random(N,S)
    
        S_null = S_alternative
        reestimate_parameters(my_A,my_B, my_pi,Y_t,halting_criteria=halting_criteria)
        S_alternative =compute_loglikelihood_parameters(my_pi, my_A, my_B, Y_t)

        model =st.chi2(df = df)
        lam = 2*(S_alternative - S_null)
        p = model.sf(lam)
        if verbose:
            print("N=",N,"df: ", df, "Lamda: ", lam, "p: ", p)
    return N-1

In [ ]:
def viterbi_algorithm(pi,A,B,Y,verbose=False):
    """Computes the most probable path for the hidden sequence (X_t)
    returns the score, and path as a list of hidden state values."""
  
    N = A.shape[0]
    T = len(Y)
    
    logA = np.log(A)
    logB = np.log(B)
    logpi = np.log(pi)
    # start with t=0
    paths = [[logpi[n]+logB[n,Y[0]],[n]] for n in range(N)]
    
    # now recurse until t = T-1
    for t in range(1, T):
        new_paths = [[] for n in range(N)]
        for n in range(N):
            scores = np.array([ paths[i][0] + logA[n,i] for i in range(N)])
            i = scores.argmax()
            new_score = scores.max() + logB[n,Y[t]]
            new_paths[n] = [new_score,paths[i][1] + [n]]

        paths = new_paths
        
    total_scores = np.array([x[0] for x in paths])
    best_score = total_scores.max()
    best_path = paths[total_scores.argmax()][1]
    return best_score, best_path

In [ ]:
class hmm():
    """This class is intended to mimic the machine learning algorithms implemented in
    Scikit-Learn. It's intended to make it as easy as possible to recover the
    parameters pi, A, B of the HMM model as well as to project the observed data into the
    hidden state space using the gamma distribution.
    
    Call the fit() method with observed data to recover the pi, A, B parameters from 
    an observed sequence of values. """
    
    def __init__(self, num_hiddenstates = None, halting_criteria=1e-6, 
                 pi = None, A = None, B = None):
        """My initializer. If you know the number of hidden states, pass it
        off via num_hiddenstates.
        
        halting_criteria is the parameter for specifying when to halt updating the values of pi, A, B.
        """
        self.N = num_hiddenstates
        self.halting_criteria = halting_criteria
        self._called_fit = False
        
        self.pi = pi
        self.A  = A
        self.B  = B
        if not self.N is None:
            if self.A is None:
                self.A = np.random.dirichlet(np.ones(self.N),size=self.N).transpose()
                
            if self.pi is None:
                self.pi = np.random.dirichlet(np.ones(self.N),size=1).flatten()
             
    def fit(self, observed, verbose=False):
        """ The method to call to find the values of pi, A, B from the observed data. 
        observed should be a list of observed values that are hashable.
        
        To map back from the index B[i,j] = Pr(Y=j | X= i) to the observed value Y,
        use the dictionary self.inv_map[j] = observed value"""
        
        unique = list(set([y for y in observed]))
        unique.sort()
        self.map = dict([(y,t) for t,y in enumerate(unique)])
        self.inv_map = dict([(t,y) for t,y in enumerate(unique)])
        self.Y_t = np.array([self.map[y] for y in observed])
        self.S = len(self.map)
        self.T = len(self.Y_t)
        
        if self.N is None:
            self.N =  determine_number_of_hidden_states(self.Y_t,
                                                        halting_criteria=self.halting_criteria,
                                                        verbose=verbose)
                
            self.pi, self.A, self.B = make_random(self.N,self.S)
        else:
            if self.B is None:
                self.B = np.random.dirichlet(np.ones(self.S),size=self.N)
                
        reestimate_parameters(self.A,self.B,self.pi,self.Y_t,
                              halting_criteria=self.halting_criteria)
        self._called_fit = True
    def transform(self):
        """Takes the observed data mapped to (Y_t) and then projected to the
        gamma sequence. Also computes the log likelihood of the observed data.
        Returns the gamma sequence. """
        
        if not self._called_fit:
            raise Exception("You need to call the fit method prior to calling the transform method.")
        
        alpha = np.zeros((self.T,self.N))
        beta  = np.zeros((self.T,self.N))
        gamma = np.zeros((self.T,self.N))
        sigma = np.zeros(self.T)
        gammaoneshot(alpha,beta,gamma,self.pi,self.A,self.B,self.Y_t,sigma)
        self.logp = compute_loglikelihood(sigma)
        return gamma
    def fit_transform(self,observed,verbose=False):
        """Fits the model to the observed data and returns the projection of the observed
        data into the hidden state space via the gamma sequence, Y_t -> gamma_t"""
        
        self.fit(observed,verbose=verbose)
        return self.transform()
    
    def viterbi(self):
        """Computes the most probable sequence of hidden states and returns the sequence """

        best_score, path = viterbi_algorithm(self.pi, self.A,self.B,self.Y_t)
        return path
    
    def compute_loglikelihood(self):
        """ Computes log Pr(Y_0,...,Y_T| pi, A, B) """
        self.logp = compute_loglikelihood_parameters(self.pi,self.A,self.B,self.Y_t)
        return self.logp

## Example usage:

### Constructing data for a HMM:
How to construct data for a Hidden Markove Model with  3 hidden states, 26 observable states and an observed sequence of 10,000 symbols:

In [ ]:
N = 3
S = 26
T = 10000

pi,A,B =  make_random(N,S)
X_t,Y_t = generate_data(pi,A,B,T=T)

### Analyzing a sequence

Here we're going to try to model with a sequence of text as being generated by a 2 hidden state HMM. The hidden states are *vowels* and *consonants*.
Part of the issue here though is that the content also includes white space and non-alphabetic characters. How will the HMM assign them to clusters?

In [ ]:
sample_data = """bash(1) general commands manual bash(1)name bash - gnu bourne-again shellsynopsis bash [options] 
[command_string | file]copyright bash is copyright (c) 1989-2018 by the free software foundation, inc.description 
bash is an sh-compatible command language interpreter that executes commands read from the standard input or from 
a file. bash also incor‐ porates useful features from the korn and c shells (ksh and csh). bash is intended to be 
a conformant implementation of the shell and utilities portion of the ieee posix specification (ieee standard 
1003.1). bash can be configured to be posix-conformant by default.options all of the single-character shell options 
documented in the description of the set builtin command, including -o, can be used as options when the shell is 
invoked. in addition, bash interprets the following op‐ tions when it is invoked: -c if the -c option is present, 
then commands are read from the first non-option argument command_string. if there are argu‐ ments after the 
command_string, the first argument is as‐ signed to $0 and any remaining arguments are assigned to the positional 
parameters. the assignment to $0 sets the name of the shell, which is used in warning and error messages. -i if 
the -i option is present, the shell is interactive. -l make bash act as if it had been invoked as a login shell 
(see invocation below). -r if the -r option is present, the shell becomes restricted (see restricted shell below). 
-s if the -s option is present, or if no arguments remain after option processing, then commands are read from 
the standard input. this option allows the positional parameters to be set when invoking an interactive shell or 
when reading input through a pipe. -v print shell input lines as they are read. -x print commands and their 
arguments as they are executed. -d a list of all double-quoted strings preceded by $ is printed on the standard 
output. these are the strings that are sub‐ ject to language translation when the current locale is no"""

sample_data = " ".join(sample_data.split()) # Remove double white spaces.

Create a model and fit it to the data:

In [ ]:
%%time
my_model = hmm(num_hiddenstates=3)
my_model.fit(sample_data)

In [ ]:
print("log likelihood of observed data:",my_model.compute_loglikelihood())

project the observed sequence to the hidden state space, $Y_t \rightarrow \gamma_t(\_)$

In [ ]:
gamma = my_model.transform()
print("Y_t\tProjection(Y_t)")
for t in range(20):
    print("{0}\t {1:6.4f} {2:6.4f} {3:6.4f}".format(sample_data[t],*gamma[t]))
#print(gamma[:20])

Find the most probable sequence of hidden states:

In [ ]:
path = my_model.viterbi()

Compare the most probable path vs. the sequence of most probable hidden states:

In [ ]:
print("Most probable X_t sequence according to Viterbi vs. Gamma argmax")
print(path[:40])
print(list(gamma[:40].argmax(axis=1)))

In [ ]:
agreements =  (np.array(path) == gamma.argmax(axis=1)).sum()
print("Percentage agreement: {0:6.2f}%".format(100*agreements/gamma.shape[0]))

Cluster symbols using the $B$ matrix. Which symbols are *vowels* and which are *consonants*?

In [ ]:
assignments = my_model.B.argmax(axis=0)

groups = [ [] for t in range(my_model.B.shape[0])]
for i in range(my_model.B.shape[1]):
    letter = my_model.inv_map[i]
    groups[assignments[i]].append(letter)
    
for t in range(my_model.B.shape[0]):
    groups[t].sort()

print("Letter clusters:")
for t in range(my_model.B.shape[0]):
    print("Group {0}: ".format(t),"".join(groups[t]))